<a href="https://colab.research.google.com/github/Kaleab1999/Medical-Transcription-for-medicne-department/blob/main/PubMed_bert_for_MT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import re
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW, Trainer, TrainingArguments
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
df= pd.read_csv("/content/preproceed MT file.csv")
df.head()

,Unnamed: 0,transcription,medical_specialty,tokenised,stemmed,stemmed_without_stop
0,3,d m mode left atrial enlargement with left a...,Cardiovascular / Pulmonary,"['d', 'm', 'mode', 'left', 'atrial', 'enlargem...","['d', 'm', 'mode', 'left', 'atrial', 'enlarg',...",mode left atrial enlarg left atrial diamet cm ...
1,4,the left ventricular cavity size and wall thi...,Cardiovascular / Pulmonary,"['the', 'left', 'ventricular', 'cavity', 'size...","['the', 'left', 'ventricular', 'caviti', 'size...",left ventricular caviti size wall thick appear...
2,7,d echocardiogram multiple views of the heart ...,Cardiovascular / Pulmonary,"['d', 'echocardiogram', 'multiple', 'views', '...","['d', 'echocardiogram', 'multipl', 'view', 'of...",echocardiogram multipl view heart great vessel...
3,9,description normal cardiac chambers size nor...,Cardiovascular / Pulmonary,"['description', 'normal', 'cardiac', 'chambers...","['descript', 'normal', 'cardiac', 'chamber', '...",descript normal cardiac chamber size normal le...
4,11,d study mild aortic stenosis widely calcifie...,Cardiovascular / Pulmonary,"['d', 'study', 'mild', 'aortic', 'stenosis', '...","['d', 'studi', 'mild', 'aortic', 'stenosi', 'w...",studi mild aortic stenosi wide calcifi minim r...


In [ ]:
model_name = 'dmis-lab/biobert-base-cased-v1.1'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4)

Some weights of the model checkpoint at dmis-lab/biobert-base-cased-v1.1 were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification we

In [ ]:
df['medical_specialty'].value_counts()

 Cardiovascular / Pulmonary    371
 General Medicine              259
 Gastroenterology              224
 Neurology                     223
Name: medical_specialty, dtype: int64

In [ ]:
df.shape

(1077, 6)

In [ ]:
data= df[['stemmed_without_stop','medical_specialty']]
data.head()

,stemmed_without_stop,medical_specialty
0,mode left atrial enlarg left atrial diamet cm ...,Cardiovascular / Pulmonary
1,left ventricular caviti size wall thick appear...,Cardiovascular / Pulmonary
2,echocardiogram multipl view heart great vessel...,Cardiovascular / Pulmonary
3,descript normal cardiac chamber size normal le...,Cardiovascular / Pulmonary
4,studi mild aortic stenosi wide calcifi minim r...,Cardiovascular / Pulmonary


In [ ]:
data.isnull().sum()

stemmed_without_stop    0
medical_specialty       0
dtype: int64

In [ ]:
data.iloc[[6]]

,stemmed_without_stop,medical_specialty
6,histori present ill patient year old femal pre...,General Medicine


In [ ]:
X= data['stemmed_without_stop'].tolist()
y= list( data['medical_specialty'].astype('category').cat.codes)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2)

### model building

In [ ]:
train_d= tokenizer(X_train, padding= True, truncation= True, max_length=512)
test_d= tokenizer(X_test, padding= True, truncation= True, max_length=512)

In [ ]:
train_d.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

#### create dataset

In [ ]:
import torch

In [ ]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [ ]:
train_dataset = Dataset(train_d, y_train)
test_dataset = Dataset(test_d, y_test)

In [ ]:
train_dataset[5]

{'input_ids': tensor([  101,  2705, 12522,  2229,  2489, 10880,  1117,  2772,  1182,  1675,
          5178,  1214,  1385,  1653, 12477, 14791,   175, 14494,  1233,  1675,
          9712,  1200,  1403,  1395,  1160,  1285,  1107, 13782,  2225, 10880,
          2793,  4267,  8517, 14226,  1182,   190,  9324,  2047, 14441,  1107,
         11916,  1149,  4163,  3121,  3252, 11437,  8005, 14703,  6922,  8136,
          1179,  5351,  3805, 11850,  1665,  1310,  1631,  2869,  2206,  1285,
          8050, 15394,  1116,   174, 19093,  5412,  4841,  4648,  1179,  2229,
         19614,  3919, 13335,  1182, 22882,  4267, 15284,  7920,  5351,  1202,
          1162,  1763,  1143, 13328,  1117,  2772,  1182,  4267, 22377,  1204,
           177, 24312, 23826,  5194,  2875,  5351, 19073,  2455,  1246, 19614,
          1223,  2246,   181, 25509,  1197, 23609, 26405, 20362,  9712,  1200,
          1403,  1395,  2999, 11019,  1361,  1885,  2999,  1218,  5351, 10552,
          1182,  3678,  2165,  5173, 22

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import numpy as np

def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    # average= 'micro' because it's multi-class classification u can use also macro, or weight or other 
    # u shouldn't mention anything for binary classification
    recall = recall_score(y_true=labels, y_pred=pred, average='micro')
    precision = precision_score(y_true=labels, y_pred=pred, average='micro')
    f1 = f1_score(y_true=labels, y_pred=pred, average='micro')

    return { "precision": precision, "recall": recall, 'f1-score': f1, "acc": accuracy}

In [ ]:
# Define Trainer
args = TrainingArguments(
    output_dir="reslt_1",
    num_train_epochs=10,
    per_device_train_batch_size=32

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 861
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 270
  Number of trainable parameters = 108313348


In [ ]:
trainer.evaluate()